##### Install all the required packages

In [1]:
from llama_cpp import Llama
from langchain import PromptTemplate
import random
import os
import re
import time
import copy
import pickle
import json

Update the model path wrt your local

<ul>
<li>Huggingface model:<a href="https://huggingface.co/TheBloke/Llama-2-7B-GGML">model link</a></li>
<li>Checkout <b>Provided file</b></li>
<li>Go to file versions</li>
<li>Download the required model</li>
<li><b>llama-2-7b.ggmlv3.q8_0.bin</b> is the largest which is working as expected</li>
</ul>


In [2]:
model_id = '/Users/fission/Downloads/LLama-2-compressed/llama-2-7b.ggmlv3.q8_0.bin'

##### n_ctx
<ul>
<li>n_ctx is Maximum context size</li>
<li>Default value is 512</li>
<li>Please fine-tune as per the input parameters</li>
</ul>

In [3]:
local_llm = Llama(model_path=model_id, n_ctx=1024, seed=random.randint(0,1337))
local_llm

llama.cpp: loading model from /Users/fission/Downloads/LLama-2-compressed/llama-2-7b.ggmlv3.q8_0.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 1024
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_head_kv  = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 5.0e-06
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 7 (mostly Q8_0)
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =    0.08 MB
llama_model_load_internal: mem required  = 6828.73 MB (+  512.00 MB per state)
llama_new_context_with_model: k

In [4]:
def get_prompt_cleaned_format(template, data, sep="\n ", replace_words={}):
    """
    generating prompt in text format
    you can only send prompt taking as input as well
    Please modify as per requirements
    :param template: Template format
    :param data: data is in dictionary format
    :param sep: please update it as per the prompt
    :return:
    """
    try:
        prompt = PromptTemplate(
            input_variables=["information"],
            template=template)
        info = ""
        i = 1
        for key,val in data.items():
            info += key+": "
            info += val
            if i<len(data):
                info += sep
            i += 1
        info = info.strip()

        # If you want prompt can only send prompt as per the requirement without formatting
        # please modify the code as per your requirements.
        output =  prompt.format(information=info).strip()
        if len(replace_words)>0:
            for key,val in replace_words.items():
                output = output.replace(key,val)
            output = " ".join(output.split()).strip()
        return output
    except Exception as ex:
        print(f"Error while generating prompt: {ex}")
        return ""

please feel free to experiment
<a href="https://towardsdatascience.com/decoding-strategies-that-you-need-to-know-for-response-generation-ba95ee0faadc"> Model parameters for fine-tuning in detail</a>
<a href="https://michaelehab.medium.com/the-secrets-of-large-language-models-parameters-how-they-affect-the-quality-diversity-and-32eb8643e631"> Wrt params fine-tuning</a>
<ul>
    <li>temperature</li>
    <li>repeat_penalty</li>
    <li>frequency_penalty</li>
    <li>top_k</li>
    <li>top_p</li>
    <li>other parameters</li>
</ul>

<a href="https://replicate.com/replicate/oasst-sft-1-pythia-12b/versions/28d1875590308642710f46489b97632c0df55adb5078d43064e1dc0bf68117c3">Website to experiment on sample prompt & fine-tuning parameters</a>

Here I am running multiple times with same parameters and checking if the output is consistent with prompt and parameters

In [5]:
def get_output(template, data, sep="\n  ", max_tokens=256,
               temperature=0.35, local_llm=local_llm, stop_words = [],
               repeat_penalty=1.1,
               frequency_penalty=0.0,
               presence_penalty=0.0,
               top_p=0.95,
               top_k = 40,
               replace_words = {},
               debug=False):
    """
    generating output from llm output using data, prompt and input parameters
    :param template: for PromptTemplate
    :param data: dictionary format as per input requirements wrt template
    :param sep: seperator between each input parameter
    :param max_tokens: max number of output tokens to generate
    :param temperature: maximum amount of probability for generating new context
    :param local_llm: loaded llm
    :param debug: if you want to view prompt
    :param stop_words: Stop words at which the template generation would stop
    :param repeat_penalty:
    :param frequency_penalty:
    :param top_p: sampling frequency, beam search
    :param top_k: number of words to give attention
    :param debug: just to print prompt, please feel free to modify add more information to debug
    :return:
    """

    try:

        # generating prompt
        generated_template = get_prompt_cleaned_format(template, data, sep=sep, replace_words=replace_words)

        # if you enable debug would print prompt
        if debug:
            print("Prompt Template:\n",generated_template)

        # if prompt isn't empty generating text using llm
        if len(generated_template)!=0 and local_llm is not None:

            # generating context from llm and template
            output =  local_llm.create_completion(
                generated_template, max_tokens=max_tokens, temperature=temperature, repeat_penalty=repeat_penalty,
                top_p=top_p,
                frequency_penalty=frequency_penalty, presence_penalty=presence_penalty, top_k=top_k, stop=stop_words)
            CLEANR = re.compile('<.*?>')
            if 'choices' in output and len(output['choices'])>0:
                i = 0
                while i<len(output['choices']):
                    output["choices"][i]["text"] = " ".join(re.sub(CLEANR, '', output["choices"][i]["text"]).split()).strip()
                    output["choices"][i]["text"] = re.sub(r'http\S+', '', output["choices"][i]["text"])\
                        .replace("URL:","").replace("URL","").replace("HTML tags:","").replace("HTML tags","")
                    output["choices"][i]["text"] = " ".join(output["choices"][i]["text"].split()).strip()
                    # re.sub(r'http\S+', '', stringliteral)
                    i += 1
            return output
    except Exception as ex:
        print(f"Error while generating output from LLM: {ex}")
        return ""

In [6]:
def dict_save(filename, json_data):
    """
    saving dictionary as json output to a text file
    In case if you want to do grid search on various parameters and compare
    :param filename: output textfile name
    :param json_data: Input dictionary name
    :return:
    """
    try:
        with open(filename, "w") as f:
            json.dump(json_data, f)
    except Exception as ex:
        print(f"Error while saving: {ex}")
    return

<a href="https://www.amazon.in/Fire-Boltt-Phoenix-Bluetooth-Calling-Monitoring/dp/B0B3RRWSF6/ref=sr_1_6?_encoding=UTF8&content-id=amzn1.sym.5b7a4262-5691-429a-b5f0-8b6cf56ce8af&pd_rd_r=091b66bb-6941-4760-922b-358096e86e16&pd_rd_w=MwZou&pd_rd_wg=XeEjX&pf_rd_p=5b7a4262-5691-429a-b5f0-8b6cf56ce8af&pf_rd_r=H0MZD6RZSY4X01S8WCMY&qid=1693287651&sr=8-6&th=1">Amazon data used to experiment</a>

In [7]:
data_1 = {"Keywords": "Fire-Boltt Phoenix Smart Watch, 1.3 Inch display, 240 Watts Lithium Ion batteries, Bluetooth Connectivity, "
                      "Android OS, 5 Days of Battery life, Made in China",
          "Description":""}

This is general daat scientist job taken by us to evaluate

In [8]:
data_2 = {"Keywords": "Data Scientist, AI, Machine Learning, Deep Learning, NLP, RNN, LSTM, Entity Recognition, Python, Coding in Python, SQL, Data Analysis, ETL, MLOps",
 "Job Description": ""}

In [9]:
template_1 = "Generate a spellbinding Description using only the input Keywords. " \
             "Don't use Price details, HTML tags, Javascript, Programming languages, URLs, or information " \
             "not in the input Keywords to generate the Description.\n {information}"

##### Checking input prompts to LLM

In [10]:
get_prompt_cleaned_format(template_1, data_1)

"Generate a spellbinding Description using only the input Keywords. Don't use Price details, HTML tags, Javascript, Programming languages, URLs, or information not in the input Keywords to generate the Description.\n Keywords: Fire-Boltt Phoenix Smart Watch, 1.3 Inch display, 240 Watts Lithium Ion batteries, Bluetooth Connectivity, Android OS, 5 Days of Battery life, Made in China\n Description:"

In [11]:
get_prompt_cleaned_format(template_1, data_2, replace_words={"Description":"Job Description", "Price details,":"",
                                                             "Programming languages,":""})

"Generate a spellbinding Job Description using only the input Keywords. Don't use HTML tags, Javascript, URLs, or information not in the input Keywords to generate the Job Description. Keywords: Data Scientist, AI, Machine Learning, Deep Learning, NLP, RNN, LSTM, Entity Recognition, Python, Coding in Python, SQL, Data Analysis, ETL, MLOps Job Job Description:"

Fine-tuned parameters please feel free to further optimize as per the template and use case
Running the results <b>5 times</b> to check if the results are <b>optimal</b> or the <b>output varies for every run</b>

please feel free to <b>add stop_words</b> post which you find the <b>results to be repeating</b>

In [12]:
%%time
data_1_output_1 = [get_output(template_1, data_1, temperature=0.2,top_p=0.2, top_k=256, frequency_penalty=0.3,
                              # stop_words =["Keywords","###","Keywords:", "Input Keywords:"]
                              ) for i in range(0,5)]
data_1_output_1


llama_print_timings:        load time = 33854.24 ms
llama_print_timings:      sample time =   401.65 ms /   256 runs   (    1.57 ms per token,   637.37 tokens per second)
llama_print_timings: prompt eval time = 33854.15 ms /   105 tokens (  322.42 ms per token,     3.10 tokens per second)
llama_print_timings:        eval time = 96750.75 ms /   255 runs   (  379.41 ms per token,     2.64 tokens per second)
llama_print_timings:       total time = 132244.59 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time = 33854.24 ms
llama_print_timings:      sample time =   383.45 ms /   256 runs   (    1.50 ms per token,   667.62 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time = 83578.43 ms /   256 runs   (  326.48 ms per token,     3.06 tokens per second)
llama_print_timings:       total time = 85023.41 ms
Llama.generate: prefix-match hit

llama_pr

CPU times: user 39min 29s, sys: 37.2 s, total: 40min 6s
Wall time: 7min 37s



llama_print_timings:        load time = 33854.24 ms
llama_print_timings:      sample time =   393.06 ms /   256 runs   (    1.54 ms per token,   651.30 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time = 83247.67 ms /   256 runs   (  325.19 ms per token,     3.08 tokens per second)
llama_print_timings:       total time = 84690.53 ms


[{'id': 'cmpl-3a4768b5-c76f-4a2d-b3ef-dc5b257348b5',
  'object': 'text_completion',
  'created': 1694078100,
  'model': '/Users/fission/Downloads/LLama-2-compressed/llama-2-7b.ggmlv3.q8_0.bin',
  'choices': [{'text': 'The Fire-Boltt Phoenix Smart Watch is a stylish and functional device that offers a range of features to help you stay connected and organized. With its 1.3-inch display, the watch provides clear visuals for viewing notifications, messages, and other information. It also has Bluetooth connectivity, allowing you to pair it with your smartphone or other compatible devices. The Phoenix Smart Watch is powered by a 240 Watts Lithium Ion battery that can last up to five days on a single charge. Additionally, the watch runs on Android OS and comes in a range of colors, including black, white, and gold. It is made in China and has a one-year warranty from Fire-Boltt. Keywords: Fire-Boltt Phoenix Smart Watch, 1.3 Inch display, 240 Watts Lithium Ion batteries, Bluetooth Connectivit

Please add <b>Keywords: & Description:</b> to stop words as content is repeating post that

Running the results for jobs data

In [13]:
%%time
data_2_output_1 = [get_output(
    template_1, data_2, temperature=0.2,top_p=0.2, top_k=256, frequency_penalty=0.3,
    stop_words=["Job Description Template:","Job Description:"],
    replace_words={"Description":"Job Description", "Price details,":"","Programming languages,":""},)
    for i in range(0,5)]
data_2_output_1

Llama.generate: prefix-match hit

llama_print_timings:        load time = 33854.24 ms
llama_print_timings:      sample time =   313.17 ms /   256 runs   (    1.22 ms per token,   817.44 tokens per second)
llama_print_timings: prompt eval time =  9191.69 ms /    87 tokens (  105.65 ms per token,     9.47 tokens per second)
llama_print_timings:        eval time = 74476.28 ms /   255 runs   (  292.06 ms per token,     3.42 tokens per second)
llama_print_timings:       total time = 84840.40 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time = 33854.24 ms
llama_print_timings:      sample time =   364.62 ms /   256 runs   (    1.42 ms per token,   702.09 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time = 83888.68 ms /   256 runs   (  327.69 ms per token,     3.05 tokens per second)
llama_print_timings:       total time = 85211.26 ms
Llama.gene

CPU times: user 38min 11s, sys: 21.8 s, total: 38min 33s
Wall time: 6min 47s



llama_print_timings:        load time = 33854.24 ms
llama_print_timings:      sample time =   331.35 ms /   256 runs   (    1.29 ms per token,   772.61 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time = 70406.42 ms /   256 runs   (  275.03 ms per token,     3.64 tokens per second)
llama_print_timings:       total time = 71573.06 ms


[{'id': 'cmpl-8751a8cf-e71f-4375-9ec5-7c676b2f7d8c',
  'object': 'text_completion',
  'created': 1694078741,
  'model': '/Users/fission/Downloads/LLama-2-compressed/llama-2-7b.ggmlv3.q8_0.bin',
  'choices': [{'text': 'The data scientist is responsible for developing and implementing machine learning models to solve complex business problems. They work with large datasets to extract insights that can be used to improve decision-making processes. The data scientist must have a strong understanding of statistics, mathematics, and computer science in order to effectively analyze the data. They should also be able to communicate their findings clearly and concisely so that non-technical stakeholders can understand them. Job Description: Data Scientists are responsible for developing and implementing machine learning models to solve complex business problems. They work with large datasets to extract insights that can be used to improve decision-making processes. The data scientist must have 

##### Please add <b>Job Description:</b> to stop_words and further fine-tune the parameters wrt model

Run grid search on all these parameters, so you can achieve better results.
Lower the values of <b>temperature</b> & <b>top-p</b> would help you achieve data what you are expecting.
To add more creativity to context like about the company and product please feel free to fine-tune the parameters <b>max tokens, temperature, top-p and top-k</b>

Parameters for grid search
<ul>
<li>temperature</li>
<li>top_k</li>
<li>top_p</li>
<li>repetition_penalty</li>
<li>presence_penalty</li>
<li>max_tokens</li>
<li>etc</li>
</ul>

Please modify it further as per you requirements make it as a custom function with tuning parameters or etc

In [ ]:
%%time
output = {}
# data = {}
max_tokens = 256
# if len(get_prompt_cleaned_format(template_2, data_1).split())>100:
#     max_tokens = 512
complete_data = []
c_start = time.time()
print(get_prompt_cleaned_format(template_1,data_1))

for temp in [0.1, 0.15 ,0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6,]:
    j = 0
    start = time.time()
    top_k = 256
    # stop_words_1 = stop_words
    penalty = 1.1
    fequency_p = 0.3
    top_p = 0.2
    if temp>=0.4:
        penalty = 1.8
        fequency_p = 0.9
    data = {}
    data["temperature"] = temp
    data["repeat_penalty"] = penalty
    data["fequency_penality"] = fequency_p
    data["top_p"] = top_p
    data["top_k"] = top_k
    data["stop_words"] = ["Keywords","###","Keywords:", "Input Keywords:","Job Description Template:"]#["###", "\\end{code}", "```", "Keywords:", "Keywords"]#,"http:","www.","https:"]
    data["results"] = []

    while j<5:
        # data["iteration"] = j+1
        run_output = None
        run_output = get_output(template_1, data_1, temperature=temp, top_p=top_p, top_k=top_k, frequency_penalty=fequency_p, repeat_penalty=penalty, stop_words=["Keywords","###","Keywords:", "Input Keywords:","Programming Language:","HTML Tags:"])

        # temp["iteration"] = j+1

        # please feel free to add entire run_output to see why it's existing, stop : length or stop_words, etc
        data["results"].append(copy.deepcopy(run_output["choices"][0]["text"]))

        # if you want to debug
        # if j==0:
        #     print(run_output)
        del run_output
        j += 1
    print(f"Time taken for {temp} temperature and {j} loops is {time.time()-start} ")
    complete_data.append(copy.deepcopy(data))
    del data
    # print(data[i],"\n\n")
total_time = time.time()-c_start
# print(f"Time taken for all the temperatures: {total_time//60} minutes and {total_time%60} seconds")
output = {"max_tokens":max_tokens, "results":complete_data}

##### You can save it post every run so you can keep comparing if you want and stop where you like

In [ ]:
dict_save("save_results.text", output)